Here, insert some new data and apply the model.

Describe whether this would be batch or online processing. Probably batch. But can still be called as a service.

It can be calculated once in the beginning of the week for the whole cohort of the clients, saved it to a database or a Google Sheet from where it can be accessed by the call center employees. (filter predicted as worthy to call to and save them as a Google Sheet)

model monitoring should be set up - maybe a week after the predictions were made considering that all the calls were made during that week and ground truth collected. a model can be evaluated and if needed, retrained with new observations.

for redeployment: model training should also be put into script and then it can be deployed to run automatically.

TODO: a couple of tests